In [1]:
data = spark.read.option("header", "true")\
  .csv("s3://bigdatateaching/nyctaxi-2013/data/")

In [2]:
fare = spark.read.option("header", "true")\
  .csv("s3://bigdatateaching/nyctaxi-2013/fare/")

In [3]:
data.describe()

DataFrame[summary: string, medallion: string, hack_license: string, vendor_id: string, rate_code: string, store_and_fwd_flag: string, pickup_datetime: string, dropoff_datetime: string, passenger_count: string, trip_time_in_secs: string, trip_distance: string, pickup_longitude: string, pickup_latitude: string, dropoff_longitude: string, dropoff_latitude: string]

In [11]:
fare.describe()

DataFrame[summary: string, medallion: string,  hack_license: string,  vendor_id: string,  pickup_datetime: string,  payment_type: string,  fare_amount: string,  surcharge: string,  mta_tax: string,  tip_amount: string,  tolls_amount: string,  total_amount: string]

In [9]:
data.show(10)

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      CMT|        1|                 N|2013-01-01 15:11:48|2013-01-01 15:18:10|              4|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| 

In [22]:
fare.printSchema()

root
 |-- medallion: string (nullable = true)
 |--  hack_license: string (nullable = true)
 |--  vendor_id: string (nullable = true)
 |--  pickup_datetime: string (nullable = true)
 |--  payment_type: string (nullable = true)
 |--  fare_amount: string (nullable = true)
 |--  surcharge: string (nullable = true)
 |--  mta_tax: string (nullable = true)
 |--  tip_amount: string (nullable = true)
 |--  tolls_amount: string (nullable = true)
 |--  total_amount: string (nullable = true)



In [23]:
data.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [10]:
fare.show(10)

+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|           medallion|        hack_license| vendor_id|    pickup_datetime| payment_type| fare_amount| surcharge| mta_tax| tip_amount| tolls_amount| total_amount|
+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|       CMT|2013-01-01 15:11:48|          CSH|         6.5|         0|     0.5|          0|            0|            7|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|       CMT|2013-01-06 00:18:35|          CSH|           6|       0.5|     0.5|          0|            0|            7|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|       CMT|2013-01-05 18:49:41|          CSH|         5.5|         1|     0.5|          0|            0|            7|
|DFD2202EE08F7A8DC...|51EE87

In [4]:
data.createOrReplaceTempView("trip_tbl")
fare.createOrReplaceTempView("fare_tbl")

In [25]:
merge = spark.sql("""
select
t.medallion              
, t.hack_license           
, t.vendor_id              
, t.pickup_datetime        
, f.` payment_type` as payment_type           
, f.` fare_amount` as fare_amount            
, f.` surcharge` as surcharge              
, f.` mta_tax` as mta_tax             
, f.` tip_amount` as tip_amount            
, f.` tolls_amount` as tolls_amount           
, f.` total_amount` as total          
, t.rate_code              
, t.store_and_fwd_flag     
, t.dropoff_datetime       
, t.passenger_count     
, t.trip_time_in_secs      
, t.trip_distance          
, t.pickup_longitude       
, t.pickup_latitude        
, t.dropoff_longitude      
, t.dropoff_latitude      
from 
trip_tbl t
join 
fare_tbl f on 
(t.medallion = f.medallion 
and t.hack_license = f.` hack_license` 
and t.pickup_datetime = f.` pickup_datetime`)""").cache()

In [ ]:
merge.write.parquet("s3://bigdatateaching/nyctaxi-2013/merged-parquet/",mode="overwrite")